## 1. Dataset Description & Feature Explanation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (classification_report, confusion_matrix, ConfusionMatrixDisplay, 
                             roc_curve, auc, accuracy_score, f1_score, roc_auc_score,
                             average_precision_score, make_scorer)
from sklearn.base import clone
from itertools import cycle

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

# Configuration
DATE_COL = "date"
SPLIT_DATE = "2022-01-01"
RANDOM_STATE = 42

# Load Dataset
df = pd.read_csv('master_dataset_ml_ready_labelled.csv')
df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors='coerce')
df = df.sort_values(by=DATE_COL).reset_index(drop=True)

# Dataset Description
description = """
### Dataset Feature Explanation:
- **Date**: The observation date (Daily frequency).
- **CDD (Cooling Degree Days)**: TX, PA, IL, NY - Measures demand for cooling (electricity/gas) based on heat.
- **HDD (Heating Degree Days)**: TX, PA, IL, NY - Measures demand for heating (gas) based on cold.
- **contract_1_price / contract_2_price**: Front-month and second-month futures prices for natural gas.
- **spot_price**: The current physical market price for natural gas.
- **storage_bcf**: US natural gas storage levels in Billion Cubic Feet (Key supply indicator).
- **us_gas_rigs**: Number of active gas drilling rigs (Future supply indicator).
- **Time Features**: Year, Month, Day of Year, Day of Week, Quarter - Captured to model seasonality.

### Enhanced Features (Added):
- **curve_spread**: Futures curve shape (contract_2 - contract_1)
- **storage_bcf_change_7d**: 7-day storage delta
- **HDD_total / CDD_total**: Aggregated weather indicators
- **net_weather**: HDD_total - CDD_total
- **ret_1, ret_3, ret_5, ret_10**: Momentum indicators (lagged returns)

### Target (Binary):
- **PriceUp**: 1 if next day's price increases, 0 otherwise
"""
print(description)
df.head()

## 2. Feature Engineering & Exploratory Data Analysis (EDA)

This section creates engineered features (matching the SGD notebook approach) and visualizes the data:
- **Binary Target**: PriceUp (1 if next day's return > 0)
- **Curve Spread**: Futures curve shape indicator
- **Storage Change**: 7-day delta in storage levels
- **Aggregated Weather**: HDD_total, CDD_total, net_weather
- **Momentum Features**: ret_1, ret_3, ret_5, ret_10 (lagged returns)

In [ ]:
# --- Feature Engineering ---
# 1. Target: Binary Next-Day Direction (PriceUp)
df['return'] = df['spot_price'].pct_change()
df['PriceUp'] = (df['return'].shift(-1) > 0).astype(int)

# 2. Synthetic Features
# A. Curve Spread (Futures curve shape)
df['curve_spread'] = df['contract_2_price'] - df['contract_1_price']

# B. Storage Change (7-day delta)
df['storage_bcf_change_7d'] = df['storage_bcf'].diff(7)

# C. Aggregated Weather
hdd_cols = [c for c in df.columns if c.startswith('HDD_')]
cdd_cols = [c for c in df.columns if c.startswith('CDD_')]

df['HDD_total'] = df[hdd_cols].mean(axis=1)
df['CDD_total'] = df[cdd_cols].mean(axis=1)
df['net_weather'] = df['HDD_total'] - df['CDD_total']

# D. Momentum / Returns Features
df['ret_1'] = df['return'].shift(1)       # Yesterday's return
df['ret_3'] = df['return'].rolling(3).mean()
df['ret_5'] = df['return'].rolling(5).mean()
df['ret_10'] = df['return'].rolling(10).mean()

# 3. Cleanup - Drop rows with NaNs created by lags/rolling/target shift
initial_shape = df.shape
df.dropna(inplace=True)
print(f"Dropped {initial_shape[0] - df.shape[0]} rows due to lag/rolling NaN creation.")

# 1. Target Class Distribution (Binary)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
sns.countplot(x='PriceUp', data=df, palette='viridis', ax=ax1)
ax1.set_title('Distribution of Binary Target (PriceUp)', fontsize=15)
ax1.set_xlabel('PriceUp (0: Down/Neutral, 1: Up)')
ax1.set_ylabel('Frequency')

# Show class balance percentages
total = len(df)
for p in ax1.patches:
    height = p.get_height()
    ax1.annotate(f'{height/total:.1%}', (p.get_x() + p.get_width()/2., height),
                ha='center', va='bottom', fontsize=12)

# 2. Correlation Analysis (including new features)
ax2 = axes[1]
new_features = ['curve_spread', 'storage_bcf_change_7d', 'HDD_total', 'CDD_total', 
                'net_weather', 'ret_1', 'ret_3', 'ret_5', 'ret_10', 'PriceUp']
corr_new = df[new_features].corr()
sns.heatmap(corr_new, annot=True, fmt='.2f', cmap='coolwarm', center=0, ax=ax2)
ax2.set_title('Engineered Features Correlation', fontsize=14)
plt.tight_layout()
plt.show()

# 3. Supply vs Price Trend (Sample)
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='storage_bcf', y='spot_price', hue='PriceUp')
plt.title('Relationship between Storage Levels and Spot Price', fontsize=14)
plt.show()

print("\nNew Binary Target Distribution (PriceUp):")
print(df['PriceUp'].value_counts(normalize=True))

## 3. Pre-processing and Chronological Split

Using chronological (time-based) split instead of random split for proper time-series evaluation.
- Train: Data before 2022-01-01
- Test: Data from 2022-01-01 onwards

In [ ]:
# Define columns to DROP (leakage and non-predictive)
cols_to_drop = [
    DATE_COL, 
    'PriceUp',
    'spot_price',  # Drop to avoid leakage (target is derived from this)
    'price_movement_scaled', 
    'price_movement_raw', 
    'return'  # This is basically the target, just unshifted
]

# Prepare features and target
X = df.drop(columns=cols_to_drop, errors='ignore')
y = df['PriceUp']

# Ensure X is strictly numeric
X = X.select_dtypes(include=[np.number])

print(f"Final Feature Set ({X.shape[1]} features):")
print(list(X.columns))

# Chronological Split at 2022-01-01 (proper for time-series data)
mask_train = df[DATE_COL] < SPLIT_DATE
mask_test = df[DATE_COL] >= SPLIT_DATE

X_train, X_test = X[mask_train], X[mask_test]
y_train, y_test = y[mask_train], y[mask_test]

print(f"\nTrain Dates: < {SPLIT_DATE} | Shape: {X_train.shape}")
print(f"Test Dates: >= {SPLIT_DATE} | Shape: {X_test.shape}")

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nClass distribution in training set:")
print(y_train.value_counts(normalize=True))
print(f"\nClass distribution in test set:")
print(y_test.value_counts(normalize=True))

## 4: Training SVM Baseline Model (Unoptimized)

In [ ]:
# 1. Initialize SVM Baseline Model
svm_model = SVC(probability=True, random_state=RANDOM_STATE)
    
print("--- Training Baseline SVM Model ---")

# Train
svm_model.fit(X_train_scaled, y_train)

# Predict
y_pred_baseline = svm_model.predict(X_test_scaled)
y_probs_baseline = svm_model.predict_proba(X_test_scaled)[:, 1]

# Multi-Metric Evaluation
baseline_acc = accuracy_score(y_test, y_pred_baseline)
baseline_f1 = f1_score(y_test, y_pred_baseline, pos_label=1)
baseline_roc = roc_auc_score(y_test, y_probs_baseline)
baseline_pr = average_precision_score(y_test, y_probs_baseline)

print(f"\n=== SVM Baseline Results ===")
print(f"Accuracy:          {baseline_acc:.4f}")
print(f"F1 Score (Up):     {baseline_f1:.4f}")
print(f"ROC-AUC Score:     {baseline_roc:.4f}")
print(f"PR-AUC Score:      {baseline_pr:.4f}")

print("\nConfusion Matrix:")
cm_baseline = confusion_matrix(y_test, y_pred_baseline)
print(cm_baseline)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_baseline, target_names=['Down (0)', 'Up (1)']))

## 5: Hyperparameter Optimization (Optimized GridSearchCV)

**Performance optimizations applied:**
1. `probability=False` during search (3-10x faster), refit with `True` only at end
2. RBF kernel only (gamma is irrelevant for linear, wastes ~50% of combos)
3. 3-fold TimeSeriesSplit (reduced from 5 - sufficient for class project)
4. Focused grid: 27 combinations × 3 folds = **81 fits** (down from 320)

In [ ]:
# === OPTIMIZED GRIDSEARCH (Based on performance recommendations) ===
# Key optimizations:
# 1. probability=False during search (3-10x faster), refit with True at end
# 2. Focus on RBF kernel only (gamma irrelevant for linear, wastes combos)
# 3. Reduced to 3-fold TimeSeriesSplit (sufficient for class project)
# 4. Smaller, focused grid
# 5. n_jobs=2 (better than -1 on limited resources due to SVC overhead)

tscv = TimeSeriesSplit(n_splits=3)  # Reduced from 5 to 3

# Focused parameter grid (RBF only, smaller)
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01],
    'class_weight': [None, 'balanced', {0: 1.0, 1: 2.5}]
}

# Use F1 score for the minority class as our optimization metric
f1_scorer = make_scorer(f1_score, pos_label=1)

print("--- Tuning SVM via GridSearchCV (OPTIMIZED) ---")
print(f"Parameter combinations: {3 * 3 * 3} = 27")
print(f"With {tscv.n_splits} CV folds = {27 * 3} = 81 model fits")
print("(Down from 320 fits - ~75% reduction)\n")

# FAST search with probability=False
grid = GridSearchCV(
    SVC(kernel='rbf', probability=False, random_state=RANDOM_STATE),  # probability=False for speed
    param_grid,
    cv=tscv,
    scoring=f1_scorer,
    n_jobs=2,  # Limited parallelism to avoid overhead
    verbose=1,
    refit=True
)
grid.fit(X_train_scaled, y_train)

print(f"\nBest Parameters: {grid.best_params_}")
print(f"Best CV F1 Score: {grid.best_score_:.4f}")

# === REFIT best model WITH probability=True for final evaluation ===
print("\nRefitting best model with probability=True for ROC-AUC/PR-AUC...")
best_params = grid.best_params_
best_model = SVC(
    kernel='rbf',
    C=best_params['C'],
    gamma=best_params['gamma'],
    class_weight=best_params['class_weight'],
    probability=True,  # Now enable for final model
    random_state=RANDOM_STATE
)
best_model.fit(X_train_scaled, y_train)

# Evaluate on Test Set
y_pred_opt = best_model.predict(X_test_scaled)
y_probs_opt = best_model.predict_proba(X_test_scaled)[:, 1]

opt_acc = accuracy_score(y_test, y_pred_opt)
opt_f1 = f1_score(y_test, y_pred_opt, pos_label=1)
opt_roc = roc_auc_score(y_test, y_probs_opt)
opt_pr = average_precision_score(y_test, y_probs_opt)

print(f"\n=== SVM Optimized Results (Test Set) ===")
print(f"Accuracy:          {opt_acc:.4f}")
print(f"F1 Score (Up):     {opt_f1:.4f}")
print(f"ROC-AUC Score:     {opt_roc:.4f}")
print(f"PR-AUC Score:      {opt_pr:.4f}")

print("\nConfusion Matrix:")
cm_opt = confusion_matrix(y_test, y_pred_opt)
print(cm_opt)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_opt, target_names=['Down (0)', 'Up (1)']))

## 6: Comparison

In [ ]:
# Create comprehensive comparison DataFrame
comparison_data = {
    'Model': ['SVM Baseline', 'SVM Optimized'],
    'Accuracy': [baseline_acc, opt_acc],
    'F1 (Up)': [baseline_f1, opt_f1],
    'ROC-AUC': [baseline_roc, opt_roc],
    'PR-AUC': [baseline_pr, opt_pr]
}
comparison_df = pd.DataFrame(comparison_data)

# Calculate improvements
comparison_df['Acc Improvement'] = ''
comparison_df.loc[1, 'Acc Improvement'] = f"{(opt_acc - baseline_acc) * 100:+.2f}%"
comparison_df['F1 Improvement'] = ''
comparison_df.loc[1, 'F1 Improvement'] = f"{(opt_f1 - baseline_f1) * 100:+.2f}%"

print("="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)
display(comparison_df)

# Visualize Comparison - Multiple Metrics
fig, axes = plt.subplots(1, 4, figsize=(18, 5))
metrics = ['Accuracy', 'F1 (Up)', 'ROC-AUC', 'PR-AUC']
colors = ['#3498db', '#e74c3c']

for i, metric in enumerate(metrics):
    ax = axes[i]
    values = comparison_df[metric].values
    bars = ax.bar(['Baseline', 'Optimized'], values, color=colors)
    ax.set_title(metric, fontsize=12, fontweight='bold')
    ax.set_ylim(0, max(values) * 1.2)
    
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
               f'{val:.3f}', ha='center', va='bottom', fontsize=10)

plt.suptitle('SVM Baseline vs. Optimized Model Performance', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Confusion Matrix Comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax1 = axes[0]
sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=['Down (0)', 'Up (1)'], yticklabels=['Down (0)', 'Up (1)'])
ax1.set_title('Baseline SVM Confusion Matrix')
ax1.set_ylabel('Actual')
ax1.set_xlabel('Predicted')

ax2 = axes[1]
sns.heatmap(cm_opt, annot=True, fmt='d', cmap='Blues', ax=ax2,
            xticklabels=['Down (0)', 'Up (1)'], yticklabels=['Down (0)', 'Up (1)'])
ax2.set_title('Optimized SVM Confusion Matrix')
ax2.set_ylabel('Actual')
ax2.set_xlabel('Predicted')

plt.tight_layout()
plt.show()

## 7. Ablation Study: Feature Reduction Analysis

Testing two hypotheses:
1. **Regional Weather Redundancy**: Removing PA/IL weather features (keeping TX, NY)
2. **Price Level Redundancy**: Removing contract price levels (keeping curve_spread)

In [ ]:
# List of features to remove based on redundancy hypotheses
cols_to_remove = [
    'CDD_PA', 'CDD_IL', 'HDD_PA', 'HDD_IL',  # Regional weather redundancy
    'contract_1_price', 'contract_2_price'    # Price levels (keep curve_spread)
]

# Prepare the reduced dataset
X_reduced = X.drop(columns=[c for c in cols_to_remove if c in X.columns])

print(f"Original features: {X.shape[1]}")
print(f"Reduced features: {X_reduced.shape[1]}")
print(f"Removed columns: {[c for c in cols_to_remove if c in X.columns]}")
print(f"\nRemaining features: {list(X_reduced.columns)}")

# Apply same chronological split
X_train_r, X_test_r = X_reduced[mask_train], X_reduced[mask_test]

# Re-scaling the reduced feature set
scaler_r = StandardScaler()
X_train_r_scaled = scaler_r.fit_transform(X_train_r)
X_test_r_scaled = scaler_r.transform(X_test_r)

# Train optimized SVM on reduced features (using same best params)
print("\n--- Training SVM on Reduced Features ---")
model_reduced = SVC(
    kernel='rbf',
    C=best_params['C'],
    gamma=best_params['gamma'],
    class_weight=best_params['class_weight'],
    probability=True,
    random_state=RANDOM_STATE
)
model_reduced.fit(X_train_r_scaled, y_train)

# Evaluate
y_pred_red = model_reduced.predict(X_test_r_scaled)
y_probs_red = model_reduced.predict_proba(X_test_r_scaled)[:, 1]

red_acc = accuracy_score(y_test, y_pred_red)
red_f1 = f1_score(y_test, y_pred_red, pos_label=1)
red_roc = roc_auc_score(y_test, y_probs_red)
red_pr = average_precision_score(y_test, y_probs_red)

# Create results DataFrame
ablation_results = pd.DataFrame({
    'Model': ['SVM (Full Features)', 'SVM (Reduced Features)'],
    'Num Features': [X.shape[1], X_reduced.shape[1]],
    'Accuracy': [opt_acc, red_acc],
    'F1 (Up)': [opt_f1, red_f1],
    'ROC-AUC': [opt_roc, red_roc],
    'PR-AUC': [opt_pr, red_pr]
})

# Add difference row
ablation_results['Acc Diff'] = ''
ablation_results.loc[1, 'Acc Diff'] = f"{(red_acc - opt_acc) * 100:+.2f}%"
ablation_results['F1 Diff'] = ''
ablation_results.loc[1, 'F1 Diff'] = f"{(red_f1 - opt_f1) * 100:+.2f}%"

print("\n" + "="*70)
print("ABLATION STUDY RESULTS")
print("="*70)
display(ablation_results)

print("\nClassification Report (Reduced Features):")
print(classification_report(y_test, y_pred_red, target_names=['Down (0)', 'Up (1)']))

## 8. Final Comparison Visualization

In [ ]:
# Create final comparison of all 3 models
final_comparison = pd.DataFrame({
    'Model': ['SVM Baseline', 'SVM Optimized', 'SVM Reduced Features'],
    'Features': [X.shape[1], X.shape[1], X_reduced.shape[1]],
    'Accuracy': [baseline_acc, opt_acc, red_acc],
    'F1 (Up)': [baseline_f1, opt_f1, red_f1],
    'ROC-AUC': [baseline_roc, opt_roc, red_roc],
    'PR-AUC': [baseline_pr, opt_pr, red_pr]
})

print("="*80)
print("FINAL MODEL COMPARISON (ALL VARIANTS)")
print("="*80)
display(final_comparison)

# Visualization - All models comparison
fig, axes = plt.subplots(1, 4, figsize=(18, 5))
metrics = ['Accuracy', 'F1 (Up)', 'ROC-AUC', 'PR-AUC']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for i, metric in enumerate(metrics):
    ax = axes[i]
    values = final_comparison[metric].values
    bars = ax.bar(['Baseline', 'Optimized', 'Reduced'], values, color=colors)
    ax.set_title(metric, fontsize=12, fontweight='bold')
    ax.set_ylim(0, max(values) * 1.2)
    
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
               f'{val:.3f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('SVM Model Performance Comparison (Binary Classification)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Print best parameters
print("\n" + "="*80)
print("BEST GRIDSEARCHCV PARAMETERS")
print("="*80)
for param, value in grid.best_params_.items():
    print(f"  {param}: {value}")

# Summary
print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"""
This enhanced SVM notebook incorporates:
1. Binary target (PriceUp) for next-day direction prediction
2. Engineered features: curve_spread, storage_change, momentum indicators (ret_1/3/5/10)
3. Chronological train/test split at {SPLIT_DATE}
4. TimeSeriesSplit cross-validation (proper for time-series)
5. GridSearchCV optimizing F1 score (for minority class)
6. Multiple evaluation metrics: Accuracy, F1, ROC-AUC, PR-AUC
7. Ablation study with reduced features

Best performing model: {'Optimized' if opt_f1 >= red_f1 else 'Reduced Features'} SVM
Best F1 Score (Up class): {max(opt_f1, red_f1):.4f}
""")